In [1]:
##importing libraries
import pandas as pd
import numpy as np
from ebmdatalab import bq, maps, charts

In [ ]:
##extracting anticoag info as requested by user
sql = '''
SELECT
  month,
  presc.practice,
  prac.name, 
  prac.postcode,
  bnf_code AS BNFCode,
  bnf_name AS BNFName,
  SUM(items) AS TotalItems,
  SUM(actual_cost) AS TotalCost  
FROM
  ebmdatalab.hscic.normalised_prescribing_standard AS presc
JOIN
  ebmdatalab.hscic.practices prac
ON
  presc.practice = prac.code
WHERE
  (bnf_code LIKE "0208020Y0%" ##rivaroxaban1
  OR bnf_code LIKE "0208020X0%" ##dabigatran1
  OR bnf_code LIKE "0208020Z0%" ##apixaban1
  OR bnf_code LIKE "0208020AA%" ##edoxaban1
  OR bnf_code LIKE "0208020V0%") ##warfarin1
  AND
  (month >= "2018-04-01")
GROUP BY
  month,
presc.practice,
  prac.name,  
  prac.postcode,
  bnf_code,
  bnf_name
  '''

df_anticoag_jb = bq.cached_read(sql, csv_path='anticoagjb.csv')
df_anticoag_jb.head(50)

C:\Users\bmackenna\Anaconda3\lib\site-packages\pandas\io\gbq.py:149: FutureWarning: verbose is deprecated and will be removed in a future version. Set logging level in order to vary verbosity
  credentials=credentials, verbose=verbose, private_key=private_key)
